# RCNN OBJECT DETECTION DEMO



Use our trained Faster R-CNN on the LISA Traffic Signs dataset (Simply dataset by keep only 3 classes )

### Imports

In [1]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import imutils
import cv2

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display


Import the object detection module.

In [2]:
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

Patches:

# Model preparation 

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [4]:
# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = 'lisa/records/classes.pbtxt'
categoryIdx = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [5]:
PATH_TO_FROZEN_GRAPH = os.getcwd()+'/lisa/experiments/exported_model/frozen_inference_graph.pb'

For the sake of simplicity we will test on 1 image:

In [14]:
TEST_IMAGE_PATH = os.getcwd() + '/test_images/test1.png'

In [26]:
NUM_CLASSES = 3

# Detection

In [27]:
# initialize a set of colors for our class labels
COLORS = np.random.uniform(0, 255, size=(NUM_CLASSES, 3))

Load an object detection model:

In [18]:

model = tf.Graph()
with model.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

Detection and show the result

In [29]:
# create a session to perform inference
with model.as_default():
    with tf.Session(graph=model) as sess:
        # grab a reference to the input image tensor and the boxes
        # tensor
        imageTensor = model.get_tensor_by_name("image_tensor:0")
        boxesTensor = model.get_tensor_by_name("detection_boxes:0")

        # for each bounding box we would like to know the score
        # (i.e., probability) and class label
        scoresTensor = model.get_tensor_by_name("detection_scores:0")
        classesTensor = model.get_tensor_by_name("detection_classes:0")
        numDetections = model.get_tensor_by_name("num_detections:0")
        # load the image from disk
        image = cv2.imread(TEST_IMAGE_PATH)
        (H, W) = image.shape[:2]

        # check to see if we should resize along the width
        if W > H and W > 1000:
            image = imutils.resize(image, width=1000)

        # otherwise, check to see if we should resize along the
        # height
        elif H > W and H > 1000:
            image = imutils.resize(image, height=1000)

        # prepare the image for detection
        (H, W) = image.shape[:2]
        output = image.copy()
        image = cv2.cvtColor(image.copy(), cv2.COLOR_BGR2RGB)
        image = np.expand_dims(image, axis=0)
        
        # perform inference and compute the bounding boxes,
        # probabilities, and class labels
        (boxes, scores, labels, N) = sess.run(
        [boxesTensor, scoresTensor, classesTensor, numDetections],
        feed_dict={imageTensor: image})

        # squeeze the lists into a single dimension
        boxes = np.squeeze(boxes)
        scores = np.squeeze(scores)
        labels = np.squeeze(labels)
        
        # loop over the bounding box predictions
        for (box, score, label) in zip(boxes, scores, labels):
            # if the predicted probability is less than the minimum
            # confidence, ignore it
            if score < 0.5: #min_confidence
                continue

            # scale the bounding box from the range [0, 1] to [W, H]
            (startY, startX, endY, endX) = box
            startX = int(startX * W)
            startY = int(startY * H)
            endX = int(endX * W)
            endY = int(endY * H)
            # draw the prediction on the output image
            label = categoryIdx[label]
            idx = int(label["id"]) - 1
            label = "{}: {:.2f}".format(label["name"], score)
            cv2.rectangle(output, (startX, startY), (endX, endY),
            COLORS[idx], 2)
            y = startY - 10 if startY - 10 > 10 else startY + 10
            cv2.putText(output, label, (startX, y),
            cv2.FONT_HERSHEY_SIMPLEX, 0.3, COLORS[idx], 1)

        # show the output image
        cv2.imshow("Output", output)
        cv2.waitKey(0)